In [5]:
import pandas as pd
import numpy as np
import pickle
from faker import Faker

# Initialize Faker
fake = Faker()

admission_types = [
    'URGENT', 'ELECTIVE', 'EW EMER.', 'DIRECT EMER.', 'EU OBSERVATION',
    'OBSERVATION ADMIT', 'DIRECT OBSERVATION',
    'AMBULATORY OBSERVATION', 'SURGICAL SAME DAY ADMISSION'
]
admission_locations = [
    'TRANSFER FROM HOSPITAL', 'TRANSFER FROM SKILLED NURSING FACILITY',
    'INTERNAL TRANSFER TO OR FROM PSYCH', 'PHYSICIAN REFERRAL',
    'EMERGENCY ROOM', 'PACU', 'PROCEDURE SITE',
    'WALK-IN/SELF REFERRAL', 'INFORMATION NOT AVAILABLE',
    'CLINIC REFERRAL'
]
insurance_types = [
    'Medicaid', 'Medicare', 'Other'
]
languages = [
    'ENGLISH', '?'
]
marital_statuses = [
    'SINGLE', 'MARRIED', 'DIVORCED', 'WIDOWED'
]
races = [
    'BLACK/CAPE VERDEAN', 'HISPANIC/LATINO - PUERTO RICAN', 'WHITE',
    'UNKNOWN', 'OTHER', 'BLACK/AFRICAN AMERICAN',
    'HISPANIC/LATINO - SALVADORAN', 'UNABLE TO OBTAIN',
    'WHITE - OTHER EUROPEAN', 'PORTUGUESE', 'HISPANIC/LATINO - CUBAN',
    'PATIENT DECLINED TO ANSWER', 'WHITE - BRAZILIAN',
    'HISPANIC OR LATINO'
]

drugs = [
    'Midodrine', 'Multivitamins W/minerals', 'Sodium Chloride',
    'Nephrocaps', 'Nicotine Polacrilex', 'OLANZapine', 'Ondansetron',
    'OxyCODONE (Immediate Release)'
]

diagnoses_types = [
    'Urinary tract infection, site not specified',
    'Acute respiratory failure',
    'Asthma, unspecified type, unspecified',
    'Solitary pulmonary nodule',
    'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group D [Enterococcus]',
    'Dysphagia, oropharyngeal phase', 'Esophageal reflux', 'Hypoxemia',
    'Methicillin resistant pneumonia due to Staphylococcus aureus',
    'Attention deficit disorder with hyperactivity'
]

def generate_patients(n):
    patients = []
    for i in range(n):
        gender = np.random.choice(['M', 'F'])
        patients.append({
            'patient_id': i,
            'name': fake.name_male() if gender == "M" else fake.name_female(),
            'age': np.random.randint(1, 90),
            'gender': gender,
            'insurance': np.random.choice(insurance_types),
            'language': np.random.choice(languages),
            'maritalStatus': np.random.choice(marital_statuses),
            'race': np.random.choice(races),
        })
    return pd.DataFrame(patients)

def generate_staff(n):
    staff = []
    for i in range(n):
        staff.append({
            'staff_id': i,
            'name': fake.name(),
            'role': np.random.choice(['Physician', 'Nurse', 'Admin']),
            'shift_start': fake.time(),
            'shift_end': fake.time()
        })
    return pd.DataFrame(staff)

def generate_admissions(patients:pd.DataFrame):
    admissions = []
    for i in range(len(patients)):
        admissions.append({
            'adm_id': i,
            'patient_id': patients.iloc[i].patient_id,
            'admit_time': fake.date_time(),
            'discharge_time': None,
            'type': np.random.choice(admission_types),
            'location': np.random.choice(admission_locations),
            
        })
    return pd.DataFrame(admissions)

def generate_omr(admissions):
    table = []
    for i in range(len(admissions)):
        omr = {
            'omr_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'weight': np.random.randint(60, 120),
            ##'height': np.random.randint(150, 220)/ 10.0, #not used in modeling
            'bp_systolic': np.random.randint(80, 220),
            'bp_diastolic': np.random.randint(60, 120),
        }
        table.append(omr)
    return pd.DataFrame(table)

def generate_prescriptions(admissions):
    prescriptions = []
    for i in range(len(admissions)):
        prescriptions.append({
            'prescription_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'drug': np.random.choice(drugs),
        })
    return pd.DataFrame(prescriptions)

def generate_diagnoses(admissions:pd.DataFrame):
    diag = []
    for i in range(len(admissions)):
        diag.append({
            'diagnosis_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'diagnosis': np.random.choice(diagnoses_types),
        })
    return pd.DataFrame(diag)

def generate_rooms(n):
    rooms = []
    beds = []
    bed_taken = 0
    room_id = 0
    bed_id = 0
    for i in range(n):
        if i % 2 == 0:#create new room
            rooms.append({
                'room_id': room_id,
                'capacity': 1,
            })
            beds.append({
                "bed_id": bed_id,
                "room": room_id,
                "adm_id": None
            })
            room_id += 1
        else:
            room = np.random.choice(rooms)#add bed to existing room
            room["capacity"] += 1
            beds.append({
                "bed_id": bed_id,
                "room": room["room_id"],
                "adm_id": None
            })
        bed_id += 1

    return pd.DataFrame(rooms), pd.DataFrame(beds)

def assign_beds(admissions, beds):
    for i in range(len(admissions)):
        adm = admissions.iloc[i].adm_id
        bed = beds[beds.adm_id.isna()].sample(1).iloc[0,0]  #choose random empty bed
        beds.loc[bed, "adm_id"] = adm.astype(int)
        print(bed)

In [6]:
staff = generate_staff(10)
patients = generate_patients(10)
admissions = generate_admissions(patients)
admissions.discharge_time = pd.to_datetime(admissions.discharge_time)
omr = generate_omr(admissions)
prescriptions = generate_prescriptions(admissions)
diagnoses = generate_diagnoses(admissions)
rooms, beds = generate_rooms(20)
assign_beds(admissions, beds)

11
1
9
0
15
6
18
2
3
5


In [7]:
staff.head()

,staff_id,name,role,shift_start,shift_end
0,0,Taylor Brown,Nurse,08:07:51,20:10:49
1,1,Tyler Olsen,Admin,04:15:58,23:44:23
2,2,Christopher Wade,Physician,14:02:21,05:45:37
3,3,Robert Orr,Nurse,09:15:09,19:44:07
4,4,Erik Gray,Admin,23:27:42,18:38:54


In [8]:
patients.head()

,patient_id,name,age,gender,insurance,language,maritalStatus,race
0,0,Allen Parks,47,M,Medicare,ENGLISH,SINGLE,BLACK/CAPE VERDEAN
1,1,Andrea Hendricks,17,F,Medicaid,?,WIDOWED,HISPANIC/LATINO - PUERTO RICAN
2,2,Austin Nelson,53,M,Medicaid,ENGLISH,SINGLE,UNKNOWN
3,3,Jodi Lucas,57,F,Medicaid,ENGLISH,WIDOWED,PATIENT DECLINED TO ANSWER
4,4,Roger White,6,M,Medicaid,?,MARRIED,WHITE - BRAZILIAN


In [9]:
admissions

,adm_id,patient_id,admit_time,discharge_time,type,location
0,0,0,1995-05-24 23:03:34.609878,NaT,ELECTIVE,PROCEDURE SITE
1,1,1,1979-12-19 16:54:26.485571,NaT,EU OBSERVATION,WALK-IN/SELF REFERRAL
2,2,2,1997-12-07 08:50:15.821082,NaT,AMBULATORY OBSERVATION,INTERNAL TRANSFER TO OR FROM PSYCH
3,3,3,1993-10-25 05:32:43.946898,NaT,EU OBSERVATION,WALK-IN/SELF REFERRAL
4,4,4,2008-02-26 01:10:08.397771,NaT,DIRECT EMER.,PACU
5,5,5,2002-03-08 09:43:48.454110,NaT,EU OBSERVATION,CLINIC REFERRAL
6,6,6,1983-08-09 09:01:00.935035,NaT,EW EMER.,TRANSFER FROM SKILLED NURSING FACILITY
7,7,7,1972-08-27 21:56:41.011412,NaT,EW EMER.,PHYSICIAN REFERRAL
8,8,8,1977-12-09 13:48:31.097121,NaT,SURGICAL SAME DAY ADMISSION,TRANSFER FROM HOSPITAL
9,9,9,2014-05-10 01:32:28.188288,NaT,URGENT,EMERGENCY ROOM


In [10]:
admissions.iloc[0,0]

0

In [11]:
omr.head()

,omr_id,adm_id,weight,bp_systolic,bp_diastolic
0,0,0,110,119,118
1,1,1,117,185,105
2,2,2,73,104,70
3,3,3,118,87,90
4,4,4,89,187,108


In [12]:
prescriptions.head()

,prescription_id,adm_id,drug
0,0,0,Nephrocaps
1,1,1,Nicotine Polacrilex
2,2,2,OxyCODONE (Immediate Release)
3,3,3,OLANZapine
4,4,4,Ondansetron


In [13]:
diagnoses.head()

,diagnosis_id,adm_id,diagnosis
0,0,0,Hypoxemia
1,1,1,Acute respiratory failure
2,2,2,Streptococcus infection in conditions classifi...
3,3,3,Attention deficit disorder with hyperactivity
4,4,4,Hypoxemia


In [14]:
rooms

,room_id,capacity
0,0,2
1,1,4
2,2,1
3,3,1
4,4,3
5,5,3
6,6,1
7,7,3
8,8,1
9,9,1


In [15]:
beds

,bed_id,room,adm_id
0,0,0,3
1,1,0,1
2,2,1,7
3,3,1,8
4,4,2,None
5,5,1,9
6,6,3,5
7,7,1,None
8,8,4,None
9,9,4,2


In [16]:
with open('app/data/staff.csv', 'w') as f:
    staff.to_csv(f, index=False)
with open('app/data/patients.csv', 'w') as f:
    patients.to_csv(f, index=False)
with open('app/data/admissions.csv', 'w') as f:
    admissions.to_csv(f, index=False)
with open('app/data/omr.csv', 'w') as f:
    omr.to_csv('app/data/omr.csv', index=False)
with open('app/data/prescriptions.csv', 'w') as f:
    prescriptions.to_csv("app/data/prescriptions.csv", index=False)
with open("app/data/diagnoses.csv", "w") as f:
    diagnoses.to_csv("app/data/diagnoses.csv", index=False)
with open("app/data/rooms.csv", "wb") as f:
    rooms.to_csv(f, index=False)
with open("app/data/beds.csv", "wb") as f:
    beds.to_csv(f, index=False)

In [17]:
with open("app/data/admission_types.pkl", "wb") as f:
    pickle.dump(admission_types, f)
with open("app/data/admission_locations.pkl", "wb") as f:
    pickle.dump(admission_locations, f)
with open("app/data/insurance_types.pkl", "wb") as f:
    pickle.dump(insurance_types, f)
with open("app/data/languages.pkl", "wb") as f:
    pickle.dump(languages, f)
with open("app/data/marital_statuses.pkl", "wb") as f:
    pickle.dump(marital_statuses, f)
with open("app/data/races.pkl", "wb") as f:
    pickle.dump(races, f)
with open("app/data/drugs.pkl", "wb") as f:
    pickle.dump(drugs, f)
with open("app/data/diagnoses_types.pkl", "wb") as f:
    pickle.dump(diagnoses_types, f)